## Декораторы

Декораторы - это объекты, которые дополняют определения функций или классов. Они задают специальные режимы работы для простых функций и методов классов путем их помещения в добавочный уровень логики, реализованный как еще одна функция, которая обычно называется метафункцией. 

Декораторы классов дополняют определения классов. Они делают то же самое для классов, добавляя поддержку управления целыми объектами и их интерфейсами. Часто пересекаются в ролях с метаклассами. 

Соответственно, декораторы бывают:

    а). классов или функций
    б). сами либо классы, либо функции
    
Синтаксически декоратор выглядит как:

    @декоратор
    def | class

**Известные нам вещи в декораторах**

Во-первых, декораторный синтаксис поддерживает property:

In [ ]:
class Person:
    def __init__(self):
        self.__name = ''
    
    @property
    def name(self):
        return self.__name
    
    @name.setter
    def name(self, val):
        self.__name = val
        
    @name.deleter
    def name(self):
        del self.__name

А во-вторых, статические методы и методы классов:

In [ ]:
class Person:
    def __init__(self):
        self.__name = ''
    
    @property
    def name(self):
        return self.__name
    
    @name.setter
    def name(self, val):
        self.__name = val
        
    @name.deleter
    def name(self):
        del self.__name

Вообще все вещи, которые вы используете в виде:

    func = mything(func)

могут использоваться с синтаксисом декораторов, например, вспомните ipywidgets и интерактивные штуки к графикам. 

#### Декораторы функций

Прежде чем приступим к декораторам, следует вспомнить, что функция в питоне - это точно такой же объект, который может сам быть аргументом другой функции, и его можно возвращать. Получается, что любую функцию можно чем-нибудь дополнить:

In [2]:
def metafunc(func):
    print('Я ничего не делаю...')
    def inner_func(x, y):
        print('Я возвращаю неизменную функцию', func.__name__)
        return func(x, y)
    return inner_func

def func(x, y):
    print('Вот мои аргументы:', x, y)
    
func = metafunc(func)

func(1, 2)

Я ничего не делаю...
Я возвращаю неизменную функцию func
Вот мои аргументы: 1 2


Для чего нам обязательно приходится прописывать эту внутреннюю функцию? А чтобы получить доступ к нашим аргументам: попробуйте написать metafunc без нее. Получится:

    def metafunc(func):
        return func(???)
        
Откуда взять x, y? Мы можем только завести их во внутренней определяемой функции. Обратите внимание, что эти внутренние x, y никак практически не связаны с теми x, y, что мы задали в определении func: нам никто не запретит назвать их по-другому и даже передать их другое количество, но с другим количеством возникнут проблемы. Что реально происходит?

- metafunc принимает объект func, который сам по себе ничего не делает, он объект (мы его не вызываем). 
- Внутри у metafunc создается новый объект, тоже функция, который тоже ничего не делает, но принимает такое же количество параметров, как и исходная функция func, и вызывает ее внутри себя, чтобы вернуть ее результат. Должен вызывать, точнее говоря, когда мы его самого вызовем. 
- Содержимое переменной func перезаписывается этой новой функцией inner_func, которую мы вернули в metafunc. То есть, metafunc - это создатель функции, а inner_func добавляет какие-то действия к вызову func. 

Таким образом, в декораторе metafunc мы переопределяем нашу функцию, добавляя к ней чего-нибудь. Для чего это нужно? Как раз в таких случаях, когда нам нужно добавить подобную логику к куче разных функций и мы не хотим ее копипастить (а еще эти функции хотим использовать иногда и без этой логики). 

Таким образом, наш декоратор для проверки деления на ноль может приписываться к самым разным функциям, где такое деление используется. 

In [6]:
# closure, замыкание

def guard_zero(operate):
    def inner(x, y):  
        if y == 0:
            print('Cannot divide by zero')
            return
        return operate(x, y)
    return inner

@guard_zero
def true_divide(x, y):
    return x / y

@guard_zero
def divide(x, y):
    return x // y

**Вложенные декораторы**

Иногда (редко) нам может захотеться написать такой декоратор, который будет принимать какие-то параметры, то есть, по сути, создать фабрику декораторов, которая будет их штамповать наподобие такой штуки:

In [ ]:
def func_factory(k, b):
    return lambda x: k * x + b

Функция func_factory сама по себе ничего не делает, но штампует функции для описания прямых линий с самыми разными коэффициентами. 

Фабрика декораторов будет делать то же самое. Давайте сразу рассмотрим рабочий пример, какой у нас был на семинаре:

In [18]:
def decodedecorator(dataType, message1, message2):
    '''это фабрика декораторов, верхний уровень. Она штампует декораторы для проверки на тип данных'''
    def decorator(func):
        '''это шаблон будущего декоратора: у него меняются его k и b - сообщение об ошибке и тип данных'''
        print(message1)
        def wrapper(*args, **kwargs):
            print(message2)
            if all([type(arg) == dataType for arg in args]):   # all = все True, any = хотя бы одно True type(arg) == tuple for arg in args
                return func(*args, **kwargs)
            return "Invalid input"
        return wrapper
    return decorator

# передавая тип данных "строка", мы проверяем аргументы нашей функции stringJoin на строковость
@decodedecorator(str, "Decorator for concatenation", "stringJoin started...")
def stringJoin(*args):
    res = ''
    for arg in args:
        res += arg
    return res

# а здесь, наоборот, на интовость
@decodedecorator(int, "Decorator for summation", "summation started...")
def summation(*args):
    res = 0
    for arg in args:
        res += arg
    return res

Decorator for concatenation
Decorator for summation


In [20]:
stringJoin('1', '2', '3')

stringJoin started...


'123'

In [21]:
summation(1, 2, 3)

summation started...


6

Получается, нам не нужно писать два отдельных декоратора для проверки на строки и на инты: profit! 

**Декораторы-классы**

Декоратор может выглядеть и как класс, тогда основную нагрузку несет его метод \_\_call\_\_, а какие-то данные могут храниться в атрибутах. 

In [23]:
class tracer:
    '''декоратор-класс для функции'''
    def __init__(self, func):
        self.calls = 0
        self.func = func
        
    def __call__(self, *args):
        self.calls += 1
        print(f'# of calls: {self.calls}, function: {self.func.__name__}')
        return self.func(*args)
    
@tracer
def func(a, b, c):
    print(a, b, c)
    
# func = tracer(func)

In [26]:
func(1, 6, 3)

# of calls: 3, function: func
1 6 3


То есть, по сути, когда мы его используем, в переменную с именем нашей функции func на самом деле кладется экземпляр класса tracer, который и будет вызываться вместо функции. 

#### Декораторы классов

По сути, делаем мы все то же самое. Для чего они могут быть нужны? Ну например, мы хотим, чтобы в каждую единицу времени существовал только один экземпляр нашего класса. Декоратор способен перехватывать вызовы имени класса при создании объектов, поэтому самое время его применить (код взят из учебника Лутца):

In [3]:
instances = {}

def singleton(aClass):
    def onCall(*args, **kwargs):
        if aClass not in instances:
            # если мы еще не создавали такой объект
            instances[aClass] = aClass(*args, **kwargs)
        # а если создавали, то вернем уже созданный
        return instances[aClass]
    return onCall

@singleton
class Person:
    def __init__(self, name):
        self.name = name
        
@singleton
class Tokenizer:
    def __init__(self):
        self.tokens = []
        
p = Person('Вася')
f = Person('Петя')
print(p.name, f.name)

Вася Вася


В действительности, когда мы пишем Person('Петя'), вызывается не конструктор класса, а функция onCall, которая и возвращает нам уже существующий объект. Класс Tokenizer() я не задействовала, но причина не желать создавать много экземпляров классов одновременно может быть понятной: если в атрибутах класса содержится гигантский список...

**Декоратор dataclass**

Это вообще отличная вещь, о которой советую просто почитать [статью на хабре](https://habr.com/ru/post/415829/). Примеры я брала оттуда. 